# ETL Movies Dataset

Para crear la API desarrollaremos un ETL básico para que nuestro producto funcione sin errores

In [32]:
import pandas as pd

In [33]:
# Leyendo el dataset
df_movies = pd.read_csv('movies_dataset.csv')
df_movies.head(2)

/tmp/ipykernel_8030/438868271.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [34]:
df_credits = pd.read_csv('credits.csv')
df_credits.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [35]:
# Unimos los dos df's con la función merge
df_movies['id'] = df_movies['id'].astype(str)
df_credits['id'] = df_credits['id'].astype(str)

df = pd.merge(df_movies, df_credits, on='id')
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [36]:
duplicates_movies = df[df.crew.duplicated(keep=False)]

print(len(df))
print("Contenido de índices duplicados en df_movies:")
print(duplicates_movies['title'][2:22])

45538
Contenido de índices duplicados en df_movies:
635                                        Happy Weekend
661                                           Tarantella
676                                          The Promise
677                                          The Promise
678                                          The Promise
679                                          The Promise
714                 War Stories Our Mother Never Told Us
841                                                 Wife
842                                                 Wife
843                                                 Wife
844                                                 Wife
869                                          Baton Rouge
955                                   A Farewell to Arms
956                                   A Farewell to Arms
957                                   A Farewell to Arms
958                                   A Farewell to Arms
1116                                

Luego de revisar el nuevo dataset encontramos duplicados que contienen la misma información, así que podemos eliminarlos sin problemas

In [37]:
# Borramos índices duplicados
df.drop_duplicates(subset='id', inplace=True)
print(len(df))

45432


In [38]:
# Quitamos columnas que no nos van a servir
df.drop(axis=1, columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'tagline', 'status', 'spoken_languages', 'belongs_to_collection', 'production_companies', 'production_countries', 'overview', 'original_language'], inplace=True)

In [39]:
# Rellenamos los valores nulos del tiempo de la película, el presupuesto y los ingresos que generó
df[['revenue', 'budget', 'runtime', 'popularity']] = df[['revenue', 'budget', 'runtime', 'popularity']].fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 45537
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   budget        45432 non-null  object 
 1   genres        45432 non-null  object 
 2   id            45432 non-null  object 
 3   popularity    45432 non-null  object 
 4   release_date  45345 non-null  object 
 5   revenue       45432 non-null  float64
 6   runtime       45432 non-null  float64
 7   title         45429 non-null  object 
 8   vote_average  45429 non-null  float64
 9   vote_count    45429 non-null  float64
 10  cast          45432 non-null  object 
 11  crew          45432 non-null  object 
dtypes: float64(4), object(8)
memory usage: 4.5+ MB


In [40]:
# Creamos un diccionario que almacenara el género, otro para los y otro para los directores
genres_dict = {}
actors_dict = {}
directors_dict = {}

In [41]:
# Creamos una función para convertir las listas de texto en un arreglo de ids para poder acceder a ellos

def decoder(row, wanted, id_dict, crew = False):
    wanted_list = []
    arr = eval(row)

    if crew == True:
        for item in arr:
            if item['job'] == 'Director':
                wanted_list.append(item[wanted])
                id_dict[item['id']] = item['name'].strip()

    else:
        for item in arr:
            wanted_list.append(item[wanted])
            id_dict[item['id']] = item['name'].strip()

    return wanted_list

In [42]:
# Desglosamos los datos para tener una lista con los géneros
df['genres_id'] = [decoder(i, 'id', genres_dict) for i in df['genres']]
df['genres_id']

0        [16, 35, 10751]
1        [12, 14, 10751]
2            [10749, 35]
3        [35, 18, 10749]
4                   [35]
              ...       
45533        [18, 10751]
45534               [18]
45535       [28, 18, 53]
45536                 []
45537                 []
Name: genres_id, Length: 45432, dtype: object

In [43]:
# Desglosamos crew para obtener a los actores mediante su id
df['actors_id'] = [decoder(i, 'id', actors_dict) for i in df['cast']]
df['actors_id']

0        [31, 12898, 7167, 12899, 12900, 7907, 8873, 11...
1        [2157, 8537, 205, 145151, 5149, 10739, 58563, ...
2             [6837, 3151, 13567, 16757, 589, 16523, 7166]
3        [8851, 9780, 18284, 51359, 66804, 352, 87118, ...
4        [67773, 3092, 519, 70696, 59222, 18793, 14592,...
                               ...                        
45533                           [240240, 1749839, 1619957]
45534    [1043186, 111636, 1204271, 278923, 1042953, 57...
45535    [23764, 2059, 46277, 1736, 58646, 54649, 55270...
45536           [544742, 1090923, 1136422, 1261758, 29199]
45537                                                   []
Name: actors_id, Length: 45432, dtype: object

In [44]:
# Desglosamos crew para obtener los crew_id de los directores
df['directors_id'] = [decoder(i, 'id', directors_dict, crew=True) for i in df['crew']]
df['directors_id']

0           [7879]
1           [4945]
2          [26502]
3           [2178]
4          [56106]
           ...    
45533    [1182809]
45534    [1051381]
45535      [67753]
45536    [1085341]
45537    [1344495]
Name: directors_id, Length: 45432, dtype: object

In [45]:
# Nos aseguramos que estén completos los diccionarios
print(len(genres_dict), len(directors_dict), len(actors_dict))

20 19895 206158


In [46]:
# Ahora podemos eliminar las columnas genres, cast y crew
df.drop(['genres', 'cast', 'crew'], axis=1, inplace=True)

In [47]:
# Eliminamos filas con valores nulos en la columna release_date
df.dropna(subset=['release_date'], inplace=True)

In [48]:
import re

# Verificamos el formato AAAA-mm-dd con este patrón
patron = r'^\d{4}-\d{2}-\d{2}$'

# Creamos una máscara booleana para identificar las filas que no cumplen con el formato
mascara = df['release_date'].apply(lambda x: re.match(patron, str(x)) is None)

# Eliminar las filas que no cumplen con el formato
df = df.drop(df[mascara].index)

In [49]:
import calendar

# Creamos una función que lea la fecha y nos de el día de la semana en español para próximamente crear la función cantidad_filmaciones_dia

def spanish_day(date):
    year, month, day = map(int, date.split('-'))
    weekday = calendar.weekday(year, month, day)
    day_name = calendar.day_name[weekday]
    day_name_spanish = {
        'Monday': 'lunes',
        'Tuesday': 'martes',
        'Wednesday': 'miercoles',
        'Thursday': 'jueves',
        'Friday': 'viernes',
        'Saturday': 'sabado',
        'Sunday': 'domingo'
    }

    return day_name_spanish.get(day_name, '')

In [50]:
# Ahora creamos nuevas columnas para el año, mes y día
release_year = []
release_month = []
release_day = []

for i in df.release_date:
    release_year.append(int(i[:4]))
    release_month.append(int(i[5:7]))
    release_day.append(spanish_day(i))

df['release_year'] = release_year
df['release_month'] = release_month
df['release_day'] = release_day

In [51]:
# La columna budget la leyó como str, así que la convertimos a float
df['budget'] = df['budget'].astype(float)

In [52]:
# Por último calculamos una columna retorno dividiendo revenue entre budget
def calculate_return(row):
    if row['budget'] != 0:
        return row['revenue'] / row['budget']
    else:
        return 0

df['return'] = df.apply(calculate_return, axis=1)

In [53]:
# Convertimos popularity a float
df['popularity'] = df['popularity'].astype(float)

In [54]:
# Convertimos el id a entero
df['id'] = df['id'].astype(int)

In [55]:
# Verificamos que el dataset esté limpio
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 0 to 45537
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   budget         45345 non-null  float64
 1   id             45345 non-null  int64  
 2   popularity     45345 non-null  float64
 3   release_date   45345 non-null  object 
 4   revenue        45345 non-null  float64
 5   runtime        45345 non-null  float64
 6   title          45345 non-null  object 
 7   vote_average   45345 non-null  float64
 8   vote_count     45345 non-null  float64
 9   genres_id      45345 non-null  object 
 10  actors_id      45345 non-null  object 
 11  directors_id   45345 non-null  object 
 12  release_year   45345 non-null  int64  
 13  release_month  45345 non-null  int64  
 14  release_day    45345 non-null  object 
 15  return         45345 non-null  float64
dtypes: float64(7), int64(3), object(6)
memory usage: 5.9+ MB


In [56]:
df.head()

,budget,id,popularity,release_date,revenue,runtime,title,vote_average,vote_count,genres_id,actors_id,directors_id,release_year,release_month,release_day,return
0,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,"[16, 35, 10751]","[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...",[7879],1995,10,lunes,12.451801
1,65000000.0,8844,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,"[12, 14, 10751]","[2157, 8537, 205, 145151, 5149, 10739, 58563, ...",[4945],1995,12,viernes,4.043035
2,0.0,15602,11.712900,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0,"[10749, 35]","[6837, 3151, 13567, 16757, 589, 16523, 7166]",[26502],1995,12,viernes,0.000000
3,16000000.0,31357,3.859495,1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0,"[35, 18, 10749]","[8851, 9780, 18284, 51359, 66804, 352, 87118, ...",[2178],1995,12,viernes,5.090760
4,0.0,11862,8.387519,1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,[35],"[67773, 3092, 519, 70696, 59222, 18793, 14592,...",[56106],1995,2,viernes,0.000000


In [57]:
# Al ver que está limpio, podemos exportarlo el dataset limpio
df.to_csv('movies.csv', index=False)

In [58]:
# Convertimos los diccionarios de genres, actors y directors en df para poder exportarlos
df_genres = pd.DataFrame([[key, value.lower()] for key, value in genres_dict.items()], columns=['id', 'genre'])
df_genres

,id,genre
0,16,animation
1,35,comedy
2,10751,family
3,12,adventure
4,14,fantasy
5,10749,romance
6,18,drama
7,28,action
8,80,crime
9,53,thriller


In [59]:
# Convertimos los diccionarios de genres, actors y directors en df para poder exportarlos
df_directors = pd.DataFrame([[key, value.lower()] for key, value in directors_dict.items()], columns=['id', 'director'])
df_directors

,id,director
0,7879,john lasseter
1,4945,joe johnston
2,26502,howard deutch
3,2178,forest whitaker
4,56106,charles shyer
...,...,...
19890,1644440,ravi udyawar
19891,1736944,shanra j. kehl
19892,42634,aaron osborne
19893,1182809,hamid nematollah


In [60]:
# Convertimos los diccionarios de genres, actors y directors en df para poder exportarlos
df_actors = pd.DataFrame([[key, value.lower()] for key, value in actors_dict.items()], columns=['id', 'actor'])
df_actors

,id,actor
0,31,tom hanks
1,12898,tim allen
2,7167,don rickles
3,12899,jim varney
4,12900,wallace shawn
...,...,...
206153,169498,jeremy lelliott
206154,1177500,jason widener
206155,93930,kiko ellsworth
206156,1686379,darrell dubovsky


In [61]:
# Exportamos los datasets de género, actores y directores
df_genres.to_csv('genres.csv', index=False)
df_actors.to_csv('actors.csv', index=False)
df_directors.to_csv('directors.csv', index=False)

### Y listo, con estos datasets limpios podemos proceder a armar la API